In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.text('p_data_source', '')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
name_schema = StructType(
    [
        StructField('forename', StringType(), True),
        StructField('surname', StringType(), True)
    ]
)

In [0]:
drivers_schema = StructType(
    [
        StructField('driverId', IntegerType(), False),
        StructField('driverRef', StringType(), True),
        StructField('number', IntegerType(), True),
        StructField('code', StringType(), True),
        StructField('name', name_schema, True),
        StructField('dob', DateType(), True),
        StructField('nationality', StringType(), True),
        StructField('url', StringType(), True)
    ]
)

In [0]:
spark.read \
    .schema(drivers_schema) \
    .json(f'{raw_folder_path}/{v_file_date}/drivers.json') \
    .withColumnRenamed('driverId', 'driver_id') \
    .withColumnRenamed('driverRef', 'driver_ref') \
    .withColumn('ingestion_date', current_timestamp()) \
    .withColumn('name', concat(col('name.forename'), lit(' '), col('name.surname'))) \
    .drop(col('url')) \
    .withColumn('data_source', lit(v_data_source)) \
    .withColumn('file_date', lit(v_file_date))\
    .write.mode('overwrite').format('parquet').saveAsTable('f1_processed.drivers')

In [0]:
dbutils.notebook.exit('Success')